In [1]:
from bs4 import BeautifulSoup
import urllib, re, pandas as pd

## 1. Colnames

In [2]:
# first page
first = urllib.urlopen('http://wwe2.osc.state.ny.us/transparency/contracts/contractresults.cfm?PageNum_rsContract=1&sb=a&a=Z0000&ac=&v=%28Enter+Vendor+Name%29&vo=B&cn=&c=-1&m1=0&y1=0&m2=0&y2=0&am=0&b=Search&order=VENDOR_NAME&sort=ASC')
soup = BeautifulSoup(first, "lxml")

colnames = []
th = soup.find_all('th')
for name in th:
    colnames.append(name.find_all('a')[1].text)
colnames

[u'Vendor Name',
 u'Department/Facility',
 u'Contract Number',
 u'Current Contract Amount',
 u'Spending to Date',
 u'Contract Start Date',
 u'Contract End Date',
 u'Contract Description',
 u'Contract Type',
 u'Original Contract Approved/Filed Date']

## 2. Parse ALL Historical Data

#### There're 2613 pages (1-2613) so far [April 8th].

In [3]:
# Order those records by different column. Remember to change their urls.
def url(page_num):
    base1 = 'http://wwe2.osc.state.ny.us/transparency/contracts/contractresults.cfm?PageNum_rsContract='
    base2 = '&sb=a&a=Z0000&ac=&v=%28Enter+Vendor+Name%29&vo=B&cn=&c=-1&m1=0&y1=0&m2=0&y2=0&am=0&b=Search&order=VENDOR_NAME&sort=ASC' 
    return base1 + str(page_num) + base2
# Test
# url(2)

In [4]:
df = pd.DataFrame(columns=colnames)
pages_range = range(1,2) # Choose the range of pages you want to parse; all -> range(1, 2614)

for i in pages_range:
    page_url = url(i)
    page = urllib.urlopen(page_url)
    soup = BeautifulSoup(page, "lxml")
    page_data = []
    rows = soup.find('table').find_all('tr')
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        page_data.append([ele for ele in cols if ele]) # why if TRUE?
    temp = pd.DataFrame(page_data, columns=colnames)
    df = df.append(temp, ignore_index=True)
    
# output should be a csv generated by pd.DataFrame

## 3. Find Upate or Append data (from website)

#### ID means 'Contract Number'<br/>Historical data (eg. csv fetched before) & web parser for 'new' data in [Open Book New York](http://wwe2.osc.state.ny.us/transparency/contracts/contractresults.cfm?PageNum_rsContract=1&sb=a&a=Z0000&ac=&v=%28Enter+Vendor+Name%29&vo=B&cn=&c=-1&m1=0&y1=0&m2=0&y2=0&am=0&b=Search&order=VENDOR_NAME&sort=ASC) 

In [5]:
# test Update function: change value by hand
df.iloc[0,0] = None
print df.iloc[0,0]

None


In [6]:
# input: historical data
# test: copy of historical data for manipulatin
test = df.copy()
id_exist = map(lambda x: str(x), test['Contract Number'].values)
test['hash'] = test.apply(lambda x: hash(tuple(x)), axis = 1)

update = []
append = []
pages_range = range(1,3) #Choose the range of pages you want to check, then append or update

for i in pages_range:
    page_url = url(i)
    page = urllib.urlopen(page_url)
    soup = BeautifulSoup(page, "lxml")
    rows = soup.find('table').find_all('tr')
    for row in rows[1:]:
        cols = row.find_all('td')
        content = [ele.text.strip() for ele in cols]
        if not str(content[2]) in id_exist:
            append.append(content)
        else:
            target_id = str(content[2])
            old_hash = test[test['Contract Number'] == target_id]['hash'].values
            new_hash = hash(tuple(content)) #hash function: hash value then for comparing two rows
            if old_hash != new_hash:
                update.append(content)

In [7]:
# test Update function: validate result 
print 
df.head(2)

,Vendor Name,Department/Facility,Contract Number,Current Contract Amount,Spending to Date,Contract Start Date,Contract End Date,Contract Description,Contract Type,Original Contract Approved/Filed Date
0,None,Division of Criminal Justice Services,C002136,"$816,450.00",$0.00,12/21/2010,10/14/2013,IGNITION INTERLOCK QUALIFIED MANUFACTURERS PRO...,Equipment,08/28/2013
1,1 A LIFESAFER INC,Division of Criminal Justice Services,C002132,"$10,108,800.00",$0.00,08/15/2013,07/22/2016,IGNITION INTERLOCK QUALIFIED MANUFACTURER,Equipment,10/04/2013


## 4. Update (update & append) Historical Data

In [8]:
# append
df = df.append(pd.DataFrame(append, columns=colnames), ignore_index=True)
# update
# For Update part, append updated records at the end of historical dataframe, 
# then drop historical record (duplication by  'Contract Number') fetched before.
df = df.append(pd.DataFrame(update, columns=colnames), ignore_index=True)
df = df.drop_duplicates(subset=['Contract Number'], keep='last') 
# reset index. (not same with index in historical dataframe, if there're some updated records)
df.reset_index(drop=True, inplace=True)

In [9]:
# -> it works!
df.head(2)

,Vendor Name,Department/Facility,Contract Number,Current Contract Amount,Spending to Date,Contract Start Date,Contract End Date,Contract Description,Contract Type,Original Contract Approved/Filed Date
0,1 A LIFESAFER INC,Division of Criminal Justice Services,C002132,"$10,108,800.00",$0.00,08/15/2013,07/22/2016,IGNITION INTERLOCK QUALIFIED MANUFACTURER,Equipment,10/04/2013
1,1 ACCORD SERVICES INC,SUNY at Buffalo,T000591,"$114,000.00","$113,999.80",04/01/2015,03/31/2016,CUSTODIAL SERVICE FOR GATEWAY,Contracts Not Subject to OSC Pre-Audit,05/05/2015


## Output

In [10]:
# for test
df.to_csv('./cusp_hackathon.csv',index=False) 

In [11]:
# df2 = pd.read_csv("./cusp_hackathon.csv", index_col=False)